TextPreprocessing+Word2Vec+ Deep Learning Model with LSTM

---



**Import Libraries**

In [ ]:
### For computational and random seed purpose
import numpy as np
np.random.seed(42)
#to read csv file
import pandas as pd
#To split into train and cv data
from sklearn.model_selection import train_test_split
#To compute AUROC
from sklearn.metrics import auc,roc_auc_score
#for AUROC graph
import matplotlib.pyplot as plt
#for oversampling technique
from imblearn.over_sampling import SMOTE # (https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html)
#Data is imbalanced, we need calibrated model
from sklearn.calibration import CalibratedClassifierCV
#for hyperparameter tuning and Cross-validation fold
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RepeatedStratifiedKFold
#to ignore the error message
import warnings
warnings.filterwarnings("ignore")
#for heatmap and other plotting technique
import seaborn as sns
#to strandize the real value data
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import LabelEncoder
#To create Knn model on datasets
from sklearn.neighbors import KNeighborsClassifier
#for accuracy
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
# For applying model on datasets
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier

#using simple network
from keras.models import Sequential
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
data = pd.read_csv("Modified_SQL_Dataset.csv")

In [ ]:
data.drop_duplicates(inplace=True)
data.head()

,Query,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1


In [ ]:
data.shape

(30907, 2)

In [ ]:
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import string
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import word2vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#printing some random variable
sent_0 = data['Query'].values[0]
print(sent_0)
print("="*50)

sent_2000 = data['Query'].values[2000]
print(sent_2000)
print("="*50)

sent_15000 = data['Query'].values[15000]
print(sent_15000)
print("="*50)

sent_20000 = data['Query'].values[20000]
print(sent_20000)
print("="*50)

" or pg_sleep  (  __TIME__  )  --
-6073"   )    )     )   or 9502  =  2012#
wilson@autoconstruccion.fi
  SELECT SUBSTR ( "SQL Tutorial", -5, 5 )  AS ExtractString;


In [ ]:
#remove the special Character :  https://stackoverflow.com/a/5843547/4084039
sent_15000 = re.sub('[^A-Za-z0-9]+',' ',sent_15000)
print(sent_15000)

wilson autoconstruccion fi


In [ ]:
sent_20000 = re.sub('[^A-Za-z0-9]+',' ',sent_20000)
print(sent_20000)

 SELECT SUBSTR SQL Tutorial 5 5 AS ExtractString 


In [ ]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_query = []
lemmatizer = WordNetLemmatizer()
# tqdm is for printing the status bar
for sentance in tqdm(data['Query'].values):
    sentance = re.sub('[^A-Za-z0-9]+', ' ', sentance)
    sentance = re.sub(r',', ' ', sentance)
    #https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
    tokenization = nltk.word_tokenize(sentance)
    sentance = ' '.join([lemmatizer.lemmatize(w) for w in tokenization])
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_query.append(sentance.strip())

100%|██████████| 30907/30907 [00:07<00:00, 3940.77it/s]


In [ ]:
preprocessed_query

['pg sleep time',
 'create user name identified pass123 temporary tablespace temp default tablespace user',
 '1 utl inaddr get host address select distinct table name select distinct table name rownum limit sys table limit 5',
 'select user id 1 1 1 union select 1 version 1',
 'select user id 1 1 union select 1 version 1',
 'select name syscolumns id select id sysobjects name tablename',
 'select user id 1 1 1 1',
 '1 load file char 47 101 116 99 47 112 97 115 115 119 100 1 1 1',
 'select user id 1 1 1 union select 1 version 1',
 'select user id 1 union select 1 version 1',
 '1 1',
 '',
 'admin 1 1',
 'select user id 1 1 1 1',
 '1 1',
 '1 utl inaddr get host address select distinct column name select distinct column name rownum limit tab column limit 5',
 'select user id 1 union select 1 version 1',
 'select user id 1 1 1 1',
 'select user id 1 1 1 1',
 'distinct',
 'select user id 1 1 1 1',
 '1 ascii lower substring select top 1 name sysobjects xtype u 1 1 116',
 'select user id 1 1 1

In [ ]:
print(len(preprocessed_query))

30907


In [ ]:
Embedding_dim=300

In [ ]:
import gensim
#train wrod2vec model
model = gensim.models.Word2Vec(sentences = preprocessed_query,size=Embedding_dim,window=5,workers=4,min_count=1)

#vocab_size
words = list(model.wv.vocab)
print(len(words))

37


In [ ]:
max_length = max([len(s.split()) for s in preprocessed_query])

In [ ]:
#vectorizer the text samples into 2d integer tensor
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(preprocessed_query)
sequences = tokenizer_obj.texts_to_sequences(preprocessed_query)

#pad_sequences

word_index = tokenizer_obj.word_index
print('found %s unique tokens.'%len(word_index))

query_pad = pad_sequences(sequences,maxlen=max_length)
label = data['Label'].values
print("shape of query_pad",query_pad.shape)
print("Shape of label",label.shape)

found 24489 unique tokens.
shape of query_pad (30907, 522)
Shape of label (30907,)


In [ ]:
num_words =  len(word_index)+1

In [ ]:
#pad_rev= pad_sequences(sequences, maxlen=max_length, padding='post')
#pad_rev.shape

In [ ]:
dbfile = open('glove_vectors','rb')
db = pickle.load(dbfile)

In [ ]:
 db['mallinson'].shape

(300,)

In [ ]:
#create a weight matrix for word in training doc
embedding_matrix = np.zeros((len(word_index) + 1, Embedding_dim))
for word, i in word_index.items():
  embedding_vector = db.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
print(num_words)

24490


In [ ]:
#from tensorflow.keras import utils as np_utils

In [ ]:
#Y= np_utils.to_categorical(data['Label'])  # one hot target as required by NN.
#x_train,x_test,y_train,y_test=train_test_split(pad_rev,Y,test_size=0.20,random_state=42)

In [ ]:
from tensorflow.keras.callbacks import Callback,EarlyStopping,ModelCheckpoint,LearningRateScheduler

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Flatten,Dropout
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
import time
Embedding_dim=300

model = Sequential()
model.add(Embedding(num_words,Embedding_dim,embeddings_initializer=Constant(embedding_matrix),input_length=max_length,trainable= False))
model.add(LSTM(128,return_sequences=True))

model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))

#filepath="model_save/weights-{epoch:02d}-{val_acc:.4f}.hdf5"
#checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc',verbose=1, save_best_only=True, mode='max')

#earlystopping_1 = EarlyStopping(monitor='loss',patience=1, verbose=1,mode="min")

#name = "LSTM-cnn-64*2-{}".format(int(time.time()))
#log_dir="logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/{}'.format(name))
#lrschedule = LearningRateScheduler(changeLearningRate,verbose=1)
#callbacks_list = [tensorboard_callback,earlystopping_1]
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 522, 300)          7347000   
                                                                 
 lstm (LSTM)                 (None, 522, 128)          219648    
                                                                 
 flatten (Flatten)           (None, 66816)             0         
                                                                 
 dense (Dense)               (None, 1)                 66817     
                                                                 
Total params: 7,633,465
Trainable params: 286,465
Non-trainable params: 7,347,000
_________________________________________________________________


In [ ]:
#split the data into train and test data

validation_split=0.2

indices = np.arange(query_pad.shape[0])
np.random.shuffle(indices)
query_pad = query_pad[indices]
label = label[indices]

num_validation_samples = int(validation_split*query_pad.shape[0])

X_train_pad = query_pad[:-num_validation_samples]
y_train = label[:-num_validation_samples]
X_test_pad = query_pad[-num_validation_samples:]
y_test = label[-num_validation_samples:]


In [ ]:
print("Shape of X_train_pad tensor:",X_train_pad.shape)
print("Shape of y_train tensor:",y_train.shape)
print("Shape of X_test_pad tensor:",X_test_pad.shape)
print("Shape of y_test tensor:",y_test.shape)

Shape of X_train_pad tensor: (24726, 522)
Shape of y_train tensor: (24726,)
Shape of X_test_pad tensor: (6181, 522)
Shape of y_test tensor: (6181,)


In [ ]:
model.fit(X_train_pad,y_train,epochs=5,batch_size=15,validation_data=(X_test_pad,y_test),verbose=True)

Epoch 1/5
1649/1649 [==============================] - 1167s 708ms/step - loss: 0.0666 - accuracy: 0.9804 - val_loss: 0.0679 - val_accuracy: 0.9785
Epoch 2/5
1649/1649 [==============================] - 1174s 712ms/step - loss: 0.0666 - accuracy: 0.9802 - val_loss: 0.0706 - val_accuracy: 0.9785
Epoch 3/5
1649/1649 [==============================] - 1218s 739ms/step - loss: 0.0635 - accuracy: 0.9811 - val_loss: 0.0676 - val_accuracy: 0.9783
Epoch 4/5
1649/1649 [==============================] - 1161s 704ms/step - loss: 0.0633 - accuracy: 0.9811 - val_loss: 0.0745 - val_accuracy: 0.9783
Epoch 5/5
1649/1649 [==============================] - 1248s 757ms/step - loss: 0.0627 - accuracy: 0.9810 - val_loss: 0.0701 - val_accuracy: 0.9783


In [ ]:
y_test_pred = model.predict(X_test_pad)
print(y_test_pred)

[[2.60675526e-09]
 [2.39355931e-08]
 [1.00000000e+00]
 ...
 [1.00000000e+00]
 [1.06564224e-01]
 [8.24404781e-11]]


In [ ]:
print(roc_auc_score(y_test,y_test_pred))

0.9925701314386564


### Summary of Model

Using Word2Vec with LSTM Model, We have got auc score = 0.99